#**AI in Industry**

**Author**: Jyoti Yadav

**Date**: March 2022

**Email**: jyoti.yadav@studio.unibo.it

##Preliminary Steps

In [70]:
# The libraries we will use are imported here, in case of runtime problems
import os, shutil  #  file management
import sys 
import pandas as pd  #  dataframe management
import numpy as np  #  data manipulation

it is necessary, to clean all the data that are in the DATASETS directory and wipe out all the subdirectories.

In [71]:
folder = os.getcwd()

print("Current work directory: " + str(folder))

dataset_folder = os.path.join(os.getcwd(), "Datasets")

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

for filename in os.listdir(dataset_folder):
    file_path = os.path.join(dataset_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

print("Cleaned")

Current work directory: /content
Cleaned


##Dataset Download

In [72]:
from urllib import request
import gzip

# Config
print("Current work directory: {}".format(os.getcwd()))

dataset_folder = os.path.join(os.getcwd(), "Datasets")

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

url = "https://github.com/jyotiyadav94/Ai-in-industry/raw/main/new_data/r205n12.gzip"

dataset_path = os.path.join(dataset_folder, "r205n12.gzip")

print(dataset_path)

def download_dataset(download_path: str, url: str):
    if not os.path.exists(download_path):
        print("Downloading dataset...")
        request.urlretrieve(url, download_path)
        print("Download complete!")


def extract_dataset(download_path: str, extract_path: str):
    data = pd.read_parquet('/content/Datasets/r205n12.gzip')
    print("Extraction completed!")
    return data

# Download
download_dataset(dataset_path, url)

#Extraction
df=extract_dataset(dataset_path, dataset_folder)


Current work directory: /content
/content/Datasets/r205n12.gzip
Download complete!
Extraction completed!


In [74]:
df.head(5)

,timestamp,avg:ambient,var:ambient,max:ambient,min:ambient,avg:dimm0_temp,var:dimm0_temp,max:dimm0_temp,min:dimm0_temp,avg:dimm10_temp,var:dimm10_temp,max:dimm10_temp,min:dimm10_temp,avg:dimm11_temp,var:dimm11_temp,max:dimm11_temp,min:dimm11_temp,avg:dimm12_temp,var:dimm12_temp,max:dimm12_temp,min:dimm12_temp,avg:dimm13_temp,var:dimm13_temp,max:dimm13_temp,min:dimm13_temp,avg:dimm14_temp,var:dimm14_temp,max:dimm14_temp,min:dimm14_temp,avg:dimm15_temp,var:dimm15_temp,max:dimm15_temp,min:dimm15_temp,avg:dimm1_temp,var:dimm1_temp,max:dimm1_temp,min:dimm1_temp,avg:dimm2_temp,var:dimm2_temp,max:dimm2_temp,...,max:mem_free,min:mem_free,avg:mem_shared,var:mem_shared,max:mem_shared,min:mem_shared,avg:mem_total,var:mem_total,max:mem_total,min:mem_total,avg:part_max_used,var:part_max_used,max:part_max_used,min:part_max_used,avg:pkts_in,var:pkts_in,max:pkts_in,min:pkts_in,avg:pkts_out,var:pkts_out,max:pkts_out,min:pkts_out,avg:proc_run,var:proc_run,max:proc_run,min:proc_run,avg:proc_total,var:proc_total,max:proc_total,min:proc_total,avg:swap_free,var:swap_free,max:swap_free,min:swap_free,avg:swap_total,var:swap_total,max:swap_total,min:swap_total,label,New_label
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
24,2020-06-01 00:00:00,25.575000,0.005000,25.6,25.4,32.250000,0.214286,33,32,34.0,0.0,34,34,33.0,0.0,33,33,35.250000,0.214286,36,35,36.0,0.0,36,36,35.0,0.0,35,35,36.0,0.0,36,36,33.0,0.0,33,33,33.0,0.0,33,...,314902784.0,314894016.0,0,0.0,0,0,329873088.0,0.0,329873088.0,329873088.0,27.5,0.0,27.5,27.5,577.370000,0.000000,577.37,577.37,56.120000,0.000000,56.12,56.12,0.0,0.0,0,0,1886.500000,0.500000,1887,1886,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
27,2020-06-01 00:15:00,25.560000,0.006857,25.6,25.4,32.333333,0.238095,33,32,34.0,0.0,34,34,33.0,0.0,33,33,35.133333,0.123810,36,35,36.0,0.0,36,36,35.0,0.0,35,35,36.0,0.0,36,36,33.0,0.0,33,33,33.0,0.0,33,...,314910784.0,314906624.0,0,0.0,0,0,329873088.0,0.0,329873088.0,329873088.0,27.5,0.0,27.5,27.5,616.440000,0.000000,616.44,616.44,71.150000,0.000000,71.15,71.15,0.0,0.0,0,0,1882.000000,2.000000,1883,1881,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
30,2020-06-01 00:30:00,25.600000,0.000000,25.6,25.6,32.533333,0.266667,33,32,34.0,0.0,34,34,33.0,0.0,33,33,35.266667,0.209524,36,35,36.0,0.0,36,36,35.0,0.0,35,35,36.0,0.0,36,36,33.0,0.0,33,33,33.0,0.0,33,...,314912000.0,314892800.0,0,0.0,0,0,329873088.0,0.0,329873088.0,329873088.0,27.5,0.0,27.5,27.5,602.120000,0.000000,602.12,602.12,65.230000,0.000000,65.23,65.23,0.0,0.0,0,0,1883.666667,6.333333,1886,1881,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
33,2020-06-01 00:45:00,25.600000,0.000000,25.6,25.6,32.071429,0.071429,33,32,34.0,0.0,34,34,33.0,0.0,33,33,35.142857,0.131868,36,35,36.0,0.0,36,36,35.0,0.0,35,35,36.0,0.0,36,36,33.0,0.0,33,33,33.0,0.0,33,...,314910400.0,314908544.0,0,0.0,0,0,329873088.0,0.0,329873088.0,329873088.0,27.5,0.0,27.5,27.5,573.103333,77.450533,581.45,563.91,39.693333,268.906533,56.62,23.88,0.0,0.0,0,0,1881.333333,2.333333,1883,1880,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
36,2020-06-01 01:00:00,25.571429,0.005275,25.6,25.4,32.071429,0.071429,33,32,34.0,0.0,34,34,33.0,0.0,33,33,35.000000,0.000000,35,35,36.0,0.0,36,36,35.0,0.0,35,35,36.0,0.0,36,36,33.0,0.0,33,33,33.0,0.0,33,...,314909952.0,314891968.0,0,0.0,0,0,329873088.0,0.0,329873088.0,329873088.0,27.5,0.0,27.5,27.5,582.865000,119.042450,590.58,575.15,52.035000,518.098050,68.13,35.94,0.0,0.0,0,0,1885.000000,0.000000,1885,1885,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
